In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import pickle

import json
import csv
import sys
import networkx as nx
from networkx.readwrite import json_graph

import time
import datetime
from datetime import datetime

import copy

import pickle

import uuid

import plotly.express as px

## Index Analysis

Focus even more on the index

In [3]:
# open data
with open(r'data_prep/databases/connection_corres.json', "r") as read_file:
    ent_list = json.load(read_file)
print(len(ent_list))

188


### hierarchies

need to be manually input?

In [4]:
df = pd.read_excel('data_raw/index_hierarchy.xlsx')
first_list = df['first_level'].tolist()
first_list = [i.strip() for i in first_list if i != "" and type(i) != float]

single_theme_list = []

COUNT = 0
for item in ent_list:
    if 'index_shapiro'in list(item['label'].keys()):
        index_theme_list = item['label']['index_shapiro']['themes']
        for theme in index_theme_list:
                if ":" in theme:
                    # print(theme)
                    topic = theme.split(":")[0]
                    # print("============",topic)
                    if topic and topic not in single_theme_list:
                        print(topic)
                        single_theme_list.append(topic)
                    

print(len(set(single_theme_list)), len(single_theme_list))
firstlev_theme_list = single_theme_list + first_list
print(len(set(firstlev_theme_list)), len(firstlev_theme_list))
if len(set(firstlev_theme_list)) < len(firstlev_theme_list):
    firstlev_theme_list = list(set(firstlev_theme_list))

with open('data_prep/firstlev_theme_list.pickle', 'wb') as f:
    pickle.dump(firstlev_theme_list, f)
# save as json
with open('data_prep/firstlev_theme_list.json', 'w') as f:
    json.dump(firstlev_theme_list, f, indent=4)

action(s), human
causation/causes
Elisabeth, Princess of Bohemia
motions/movement
power
souls
analogy/analogies
body, human
extension
will
matter
mind
Descartes, René
God
geometry
medicine
Epicurus
blood
judgment
individual(s)
animals
human
princes
subjects
Princess of Bohemia
25 25
249 273


## quantification based on counting, etc.

In [10]:
# based on each letter
# this is only for themes

with open('data_prep/firstlev_theme_list.pickle', 'rb') as f:
    firstlev_theme_list = pickle.load(f)

firstlev_theme_list = [i.strip() for i in firstlev_theme_list if i]

# firstlev_theme_list

with open(r'data_prep/connection_corres.json', "r") as read_file:
    ent_list = json.load(read_file)

update_ent_list = []
index_theme_dict = {}
COUNT = 0

for item in ent_list:
    if 'index_shapiro'in list(item['label'].keys()):
        COUNT += 1
        index_theme_list = item['label']['index_shapiro']['themes']
        item['label']['index_shapiro']['grouped_themes'] = {}
        group_dict = {} # topic:themes_list
        
        for s in firstlev_theme_list:
            # group_dict[s] = [] # if no corresponding themes, the topic group will be left empty
            for theme in index_theme_list:
                if theme.startswith(s):
                    # add to list if it is not in the list
                    if s not in group_dict.keys():
                        group_dict[s] = [theme]
                    else:
                        if theme not in group_dict[s]:
                            group_dict[s].append(theme)

        item['label']['index_shapiro']['grouped_themes'] = group_dict
        item['label']['index_shapiro']['grouped_themes_count'] = len(group_dict)
        item['label']['index_shapiro']['grouped_themes_count_detailed'] = {
            k: len(v) for k, v in group_dict.items()
        }
        # print(item['label']['index_shapiro']['grouped_themes_count_detailed'],"\n")
        update_ent_list.append(item) # only consider letters in Lisa Shapiro's book

if COUNT == len(update_ent_list):
    # print list of disctionaries beautifully
    print(json.dumps(update_ent_list, indent=4))
    # save to json file
    with open('data_prep/r_e_corres_theme_group.json', 'w') as f:
        json.dump(update_ent_list, f, indent = 4)

[
    {
        "type": "correspondence",
        "label": {
            "index_shapiro": {
                "themes": [
                    "action(s), human: voluntary",
                    "animal spirits",
                    "causation/causes: mechanist account(s)",
                    "causation/causes: in mind-body interaction",
                    "Elisabeth, Princess of Bohemia: concern with privacy of correspondence",
                    "voluntary action",
                    "motions/movement: capacity/power to move and be moved",
                    "motions/movement: determination of",
                    "power: of the soul to act on the body",
                    "souls: interaction with the body",
                    "souls: nature of",
                    "souls: as nonextended and immaterial",
                    "souls: as a substance",
                    "souls: as thinking",
                    "substance(s), immaterial",
                    "substance(s), separat

In [5]:
# based on each theme
with open('data_prep/firstlev_theme_list.pickle', 'rb') as f:
    firstlev_theme_list = pickle.load(f)

# note that this is a updated version 
with open(r'data_prep/r_e_corres_theme_group.json', "r") as read_file:
    ent_list = json.load(read_file)


topic_count_dict = {} # topic here includes everything: theme, people, texts
for s in firstlev_theme_list:
    info_dict = {}
    COUNT_THEME = 0
    COUNT_PEOPLE = 0
    COUNT_TEXT = 0
    for item in ent_list:
        for topic in item['label']['index_shapiro']['grouped_themes'].keys():
            if s == topic: # for each topic, record the count of themes and details
                
                info_dict[item['unique_id']] = {
                    "themes_count": len(item['label']['index_shapiro']['grouped_themes'][topic]),
                    "themes_list": item['label']['index_shapiro']['grouped_themes'][topic]
                }
                COUNT_THEME += len(item['label']['index_shapiro']['grouped_themes'][topic])
                COUNT = COUNT_THEME
        for topic in item['label']['index_shapiro']['people']:
            if s == topic: # for each topic, record the count of themes and details
                
                info_dict[item['unique_id']] = {
                    "people_count": 1, # can only be one because there is no hierarchies in people
                    "people_list": s
                }
                COUNT_PEOPLE += 1
                COUNT = COUNT_PEOPLE
        for topic in item['label']['index_shapiro']['texts']:
            if s == topic: # for each topic, record the count of themes and details
                
                info_dict[item['unique_id']] = {
                    "text_count": 1,
                    "text_list": s
                }
                COUNT_TEXT += 1
                COUNT = COUNT_TEXT
    # update information 
    topic_count_dict[s] = {
        "count_occurrence": COUNT,
        "count_letter": len(list(info_dict.keys())),
        "count_letter_detailed": info_dict
    }

# print list of disctionaries beautifully
# print(json.dumps(topic_count_dict, indent=4))

# save to json file
with open('data_prep/r_e_topic_corres_count.json', 'w') as f:
    json.dump(topic_count_dict, f, indent = 4)
        

### Basic visualization

In [6]:
# make the dataframe

with open(r'data_prep/r_e_corres_theme_group.json', "r") as read_file:
    ent_list = json.load(read_file)

df = pd.DataFrame(columns=['correspondence_id', "date", 'from', 'from','to', 'location', 'grouped_themes_count', "grouped_themes_count_detailed"]) # initialize df
df

corres_id_list = [item['unique_id'] for item in ent_list]
from_list = [item['from'] for item in ent_list]
to_list = [item['to'] for item in ent_list]
location_list = [str(item['location']) for item in ent_list]
grouped_themes_count_list = [item['label']['index_shapiro']['grouped_themes_count'] for item in ent_list]
grouped_themes_count_detailed_list = [str(item['label']['index_shapiro']['grouped_themes_count_detailed']) for item in ent_list]

df['correspondence_id'] = corres_id_list
df['from'] = from_list
df['to'] = to_list
df['location'] = location_list
df['grouped_themes_count'] = grouped_themes_count_list
df['grouped_themes_count_detailed'] = grouped_themes_count_detailed_list
# we only have one date for these letters
df["date"] = [str(item['date']) for item in ent_list]

df_topic_corres_count = copy.deepcopy(df)
# sort
df_topic_corres_count = df_topic_corres_count.sort_values(by=['grouped_themes_count'], ascending=False)

# ==================================

with open(r'data_prep/r_e_topic_corres_count.json', "r") as read_file:
    theme_count_dict = json.load(read_file)

# make the dataframe
df = pd.DataFrame(columns=['topic', 'count_occurrence', 'count_letter', 'count_letter_detailed']) # initialize df

df['topic'] = list(theme_count_dict.keys())
df['count_occurrence'] = [theme_count_dict[i]['count_occurrence'] for i in df['topic'] ]
df['count_letter'] = [theme_count_dict[i]['count_letter'] for i in df['topic'] ]
df['count_letter_detailed'] = [str(list(theme_count_dict[i]['count_letter_detailed'].keys())) for i in df['topic'] ]

df_corres_topic_count = copy.deepcopy(df)
# sort
df_corres_topic_count = df_corres_topic_count.sort_values(by=['count_occurrence'], ascending=False)



In [8]:
df_corres_topic_count

,topic,count_occurrence,count_letter,count_letter_detailed
146,passion,82,23,"['e388f0d6-7622-4de8-b7e4-24486a843000', 'cef5..."
56,souls,70,18,"['4beed4b9-b5ad-4d85-af2e-6fa2dcfbed7b', 'e388..."
66,God,54,13,"['a8f22e20-ce9b-42a9-b121-e5a32998e24e', '5713..."
67,"Allen, Thomas",54,0,[]
90,good(s),50,19,"['047098b5-60fc-4e61-a2a6-f8dd8134bf37', '4b92..."
...,...,...,...,...
89,immaterial thing,1,1,['cef5aa9c-09cd-45fc-89de-fae88a137449']
93,Princess of Bohemia,1,1,['39fb4e69-6999-4825-b28e-617501086d2a']
95,"error, rule for avoiding",1,1,['d329db9b-d4fc-48c3-b786-a65fcaa7a0ee']
192,Ptolemy,1,1,['5005ad27-1221-4f88-87eb-346718cfcd32']


In [7]:


# counting the number of letters for each topic
fig = px.bar(df_corres_topic_count, y='count_occurrence', x='topic', 
    # color='count_letter_detailed',
    labels={'count_occurrence':'Occurences of topics'}, title="The overall occurences of topics (all letters)")
            # count_letter_detailed
fig.show()
fig.write_html("visualizations/topic_count.html")

fig = px.bar(df_corres_topic_count, y='count_letter', x='topic', 
    # color='count_letter_detailed',
    labels={'count_letter':'Correspondence count'}, title="Count of letters that contain a certain topic")
            
fig.show()
fig.write_html("visualizations/corres_topic_count.html")

# counting the occurences of each topic across all letters
fig = px.bar(df_topic_corres_count, y='grouped_themes_count', x='date', 
    color='correspondence_id',
    labels={'grouped_themes_count':'Themes count'}, title="Count of topics across correspondences", height=400)
            
fig.show()
fig.write_html("visualizations/topic_corres_count.html")


# count the distribution of topics and subtopics



### Association inside each letter

What are the themes that appear together the most frequently?

## include the original texts and do comparison



## Extract other information

## Plan:

use distant reading to read more materials and generate even just the most frequent topics

## Comparison

In [74]:
len(par_theme_list), len(sub_theme_list)

(173, 507)

In [78]:
with open(r'data_prep/r_e_corres_theme_group.json', "r") as read_file:
    ent_list = json.load(read_file)

par_theme_list, sub_theme_list = [], []

for ent_dict in ent_list:
    # print(list(ent_dict['label']['index_shapiro']['grouped_themes'].values()))
    par_theme_list += list(ent_dict['label']['index_shapiro']['grouped_themes'].keys())

    for theme_list in list(ent_dict['label']['index_shapiro']['grouped_themes'].values()):
        # print(theme_list)
        sub_theme_list += theme_list

par_theme_list = list(set(par_theme_list))
sub_theme_list = list(set(sub_theme_list))

# remove certain themes
# par_theme_list.remove('Elisabeth, Princess of Bohemia')
# par_theme_list.remove('Descartes, Ren\u00e9')

# topic distribution
par_theme_count_dict = {}
for theme in par_theme_list:
    par_theme_count_dict[theme] = 0

for ent_dict in ent_list:
    for theme in list(ent_dict['label']['index_shapiro']['grouped_themes'].keys()):
        par_theme_count_dict[theme] += 1

# remove keys
par_theme_count_dict.pop('Elisabeth, Princess of Bohemia')
par_theme_count_dict.pop('Descartes, Ren\u00e9')


# subtopic distribution
sub_theme_count_dict = {}
for theme in sub_theme_list:
    sub_theme_count_dict[theme] = 0

for ent_dict in ent_list:
    for theme in list(ent_dict['label']['index_shapiro']['grouped_themes'].values()):
        for subtheme in theme:
            sub_theme_count_dict[subtheme] += 1

# visualize
df = pd.DataFrame(columns=['topic', 'count']) # initialize df
df['topic'] = list(par_theme_count_dict.keys())
df['count'] = list(par_theme_count_dict.values())

df = df.sort_values(by=['count'], ascending=False)

# only keep the top 20
df = df[:50]


# use plotly to create a bar chart
fig = px.pie(df, values='count', names='topic', title="Distribution of themes (parent topics) across all letters")
# only show the top 10
fig.update_traces(textposition='inside', textinfo='label+percent')
# change color
fig.update_traces(marker=dict(colors=px.colors.sequential.dense[::-1]*2 + px.colors.sequential.PuBu[::-1]*10))
# extend the color range to make it cover the whole range of values

# fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()


# fig = px.bar(df, y='count', x='topic',
#     labels={'count':'Count'}, title="Count of topics across correspondences", height=400)

# fig.show()


In [156]:

# bertopic

with open(r'X:/Documentation/research/SW/shapiro_correspondence/connection_corres_with_bert.json', "r") as read_file:
    ent_list = json.load(read_file)

bert_theme_list = []

for ent_dict in ent_list:
    # print(list(ent_dict['label']['index_shapiro']['grouped_themes'].values()))
    try:
        bert_theme_list += list(ent_dict['label']['bertopic']['themes'])
    except:
        pass


bert_theme_list = list(set(bert_theme_list))
sub_theme_list = list(set(sub_theme_list))

# remove certain themes
# bert_theme_list.remove('Elisabeth, Princess of Bohemia')
# bert_theme_list.remove('Descartes, Ren\u00e9')

# topic distribution
par_theme_count_dict = {}
for theme in bert_theme_list:
    par_theme_count_dict[theme] = 0

for ent_dict in ent_list:
    try:
        for theme in list(ent_dict['label']['bertopic']['themes']):
            par_theme_count_dict[theme] += 1
    except:
        pass



# visualize
df = pd.DataFrame(columns=['topic', 'count']) # initialize df
df['topic'] = list(par_theme_count_dict.keys())
df['count'] = list(par_theme_count_dict.values())

df = df.sort_values(by=['count'], ascending=False)

# only keep the top 20
df = df[:50]


# use plotly to create a bar chart
fig = px.pie(df, values='count', names='topic', title="Distribution of themes (parent topics) across all letters")
# only show the top 10
fig.update_traces(textposition='inside', textinfo='percent')
# change color
fig.update_traces(marker=dict(colors=px.colors.sequential.dense[::-1]*2 + px.colors.sequential.PuBu[::-1]*10))
# extend the color range to make it cover the whole range of values

# fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()


# fig = px.bar(df, y='count', x='topic',
#     labels={'count':'Count'}, title="Count of topics across correspondences", height=400)

# fig.show()














In [80]:
# visualize how many subtopics are there for each topic
theme_sub_count_dict = {}
for theme in par_theme_list:
    theme_sub_count_dict[theme] = 0

for ent_dict in ent_list:
    for theme in list(ent_dict['label']['index_shapiro']['grouped_themes'].keys()):
        theme_sub_count_dict[theme] += len(ent_dict['label']['index_shapiro']['grouped_themes'][theme])

theme_sub_count_dict.pop('Elisabeth, Princess of Bohemia')
theme_sub_count_dict.pop('Descartes, Ren\u00e9')

# visualize
df = pd.DataFrame(columns=['topic', 'count']) # initialize df
df['topic'] = list(theme_sub_count_dict.keys())
df['count'] = list(theme_sub_count_dict.values())

df = df.sort_values(by=['count'], ascending=False)

# only keep the top 20
df = df[:50]

# pie chart
fig = px.pie(df, values='count', names='topic', title="Distribution of subtopics across all parent topics")
# only show the top 10
fig.update_traces(textposition='inside', textinfo='label+percent')
# change color
fig.update_traces(marker=dict(colors=px.colors.sequential.dense[::-1]*2 + px.colors.sequential.PuBu[::-1]*10))
# extend the color range to make it cover the whole range of values

# fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()


In [106]:

# create a dictionary of letter and topics they contain
letter_topic_dict = {}
for ent_dict in ent_list:
    letter_topic_dict[ent_dict['unique_id']] = list(ent_dict['label']['index_shapiro']['grouped_themes'].keys())

# create a dictionary of letter and subtopics they contain
subtopic_letter_dict = {}
for ent_dict in ent_list:
    subtopic_letter_dict[ent_dict['unique_id']] = []
    for theme in list(ent_dict['label']['index_shapiro']['grouped_themes'].values()):
        subtopic_letter_dict[ent_dict['unique_id']] += theme

# create a graph
G_topic = nx.Graph()

# add nodes
# for ent_dict in ent_list:
#     G_topic.add_node(ent_dict['unique_id'])

# add edges

ent_topic_edge_list, top_subtop_edge_list = [], []


ent_id_list = list(letter_topic_dict.keys())
topic_list = copy.deepcopy(par_theme_list)
subtopic_list = copy.deepcopy(sub_theme_list)

for source in ent_id_list:
    for target in ent_id_list: # because all nodes are thinkers 
        if source != target:
            shared_ref = len(set(letter_topic_dict[source]) & set(letter_topic_dict[target]))
            # if source mentioned target
            if target in letter_topic_dict[source]:
                G_topic.add_edge(source, target, weight=1)
            if shared_ref > 0:
                G_topic.add_edge(source, target, weight=shared_ref)

# # save the graph
# pickle.dump(G_topic, open('database/graph/thinker_reference_network.pickle', 'wb'))

# visualize
%run net_vis.py


highlight_node_list_color = {
                        '#676767': topic_list,
                        '#9b51e0': ent_id_list, # #05f3f3 
                        # NOTE: what if a node is in both lists?
                        '#2fa4a8': ent_id_list
                        }
highlight_edge_list_color = {'#9b51e0': ent_topic_edge_list
                            #  '#676767': top_subtop_edge_list
                             }

vis_graph_design(G_topic, highlight_node_list_color, highlight_edge_list_color
                     )

print(len(topic_list), len(ent_id_list), len(ent_id_list))


173 61 61


In [144]:
G_topic_let = nx.Graph()

# topic_letter_dict is the reverse of letter_topic_dict
topic_letter_dict = {}
for letter in letter_topic_dict.keys():
    for topic in letter_topic_dict[letter]:
        if topic in topic_letter_dict.keys():
            topic_letter_dict[topic].append(letter)
        else:
            topic_letter_dict[topic] = [letter]

# add nodes
# for ent_dict in ent_list:
#     G_topic_let.add_node(ent_dict['unique_id'])

# add edges

ent_topic_edge_list, top_subtop_edge_list = [], []


ent_id_list = list(topic_letter_dict.keys())
topic_list = copy.deepcopy(par_theme_list)

# remove certain topics from the top_list
topic_list.remove('Elisabeth, Princess of Bohemia')
topic_list.remove('Descartes, Ren\u00e9')

subtopic_list = copy.deepcopy(sub_theme_list)

for source in topic_list:
    for target in topic_list: # because all nodes are thinkers 
        if source != target:
            shared_ref = len(set(topic_letter_dict[source]) & set(topic_letter_dict[target]))
            # if source mentioned target
            if target in topic_letter_dict[source]:
                G_topic_let.add_edge(source, target, weight=1)
                G_topic_let.nodes[source]['label'] = source
                G_topic_let.nodes[target]['label'] = target 
            if shared_ref > 0:
                G_topic_let.add_edge(source, target, weight=shared_ref)
            
                G_topic_let.nodes[source]['label'] = source
                G_topic_let.nodes[target]['label'] = target 
# # save the graph
# pickle.dump(G_topic, open('database/graph/thinker_reference_network.pickle', 'wb'))

# visualize
%run net_vis.py


highlight_node_list_color = {
                        '#676767': topic_list,
                        '#9b51e0': topic_list, # #05f3f3 
                        # NOTE: what if a node is in both lists?
                        # '#2fa4a8': topic_list
                        }
highlight_edge_list_color = {'#9b51e0': ent_topic_edge_list
                            #  '#676767': top_subtop_edge_list
                             }

vis_graph_design(G_topic_let, highlight_node_list_color, highlight_edge_list_color
                     )

print(len(topic_list), len(ent_id_list), len(ent_id_list))



171 173 173


In [107]:

# create a graph
G_subtopic = nx.Graph()

# add edges

ent_topic_edge_list, top_subtop_edge_list = [], []


ent_id_list = list(topic_letter_dict.keys())
topic_list = copy.deepcopy(par_theme_list)
subtopic_list = copy.deepcopy(sub_theme_list)

for source in ent_id_list:
    for target in ent_id_list: # because all nodes are thinkers 
        if source != target:
            shared_ref = len(set(topic_letter_dict[source]) & set(topic_letter_dict[target]))
            # if source mentioned target
            if target in topic_letter_dict[source]:
                G_subtopic.add_edge(source, target, weight=1)
            if shared_ref > 0:
                G_subtopic.add_edge(source, target, weight=shared_ref)

# # save the graph
# pickle.dump(G_topic, open('database/graph/thinker_reference_network.pickle', 'wb'))

# visualize
%run net_vis.py


highlight_node_list_color = {
                        '#676767': topic_list,
                        '#9b51e0': ent_id_list, # #05f3f3 
                        # NOTE: what if a node is in both lists?
                        '#2fa4a8': ent_id_list
                        }
highlight_edge_list_color = {'#9b51e0': ent_topic_edge_list
                            #  '#676767': top_subtop_edge_list
                             }

vis_graph_design(G_subtopic, highlight_node_list_color, highlight_edge_list_color
                     )

print(len(topic_list), len(ent_id_list), len(ent_id_list))


173 173 173
